In [1]:
import re
import string
import sqlite3
import io
import time
from datetime import timedelta
import gensim

In [2]:
#Menghubungkan ke database
from sqlite3 import Error
db_file = 'DBnews.db'
conn = None
try:
    conn = sqlite3.connect(db_file)
except Error as e:
    print(e)

In [3]:
#mengambil kolom comments untuk diolah
comments = []
cur = conn.cursor()
cur.execute("SELECT * from t_comments")
rows = cur.fetchall()
for row in rows:
    comments.append(row)

OperationalError: no such table: t_comments

In [ ]:
comments

In [ ]:
#Mengambil hanya konten comments
new_comments = []
for comment in comments:
    new_comments.append(comment[1])

In [ ]:
new_comments

In [ ]:
#Preprocessing data (removePunctuation dan lowercase)

def removePunctuation(x):
    # Lowercasing all words
    x = x.lower()
    # Removing non ASCII chars
    x = re.sub(r'[^\x00-\x7f]',r' ',x)
    # Removing (replacing with empty spaces actually) all the punctuations
    return re.sub("["+string.punctuation+"]", " ", x)

pre_comments = []
for comment in new_comments:
    pre_comments.append(removePunctuation(comment))

In [ ]:
pre_comments

In [ ]:
#Tokenisasi hasil preprocessing data
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer('\s+', gaps=True)
tok_comments = []
for comment in low_comments:
    tok_comments.append(tokenizer.tokenize(comment))

In [ ]:
tok_comments

In [ ]:
#Menghilangkan token yang kosong
for comment in tok_comments:
    for sub_comment in comment:
        if sub_comment == '':
            comment.remove(sub_comment)

In [ ]:
tok_comments

In [ ]:
#Untuk mempercepat proses, gunakan 1000 data
tok_comments = tok_comments[:1000]

In [ ]:
#Mengolah date menggunakan corpus yang sudah ada
start_time = time.time()
print('Streaming wiki...')
id_wiki = gensim.corpora.WikiCorpus('idwiki-latest-pages-articles.xml.bz2', lemmatize=False, dictionary={})
article_count = 0
with io.open('idwiki.txt', 'w', encoding='utf-8') as wiki_txt:
    for text in id_wiki.get_texts():
        wiki_txt.write(" ".join(text) + '\n')
        article_count += 1
        if article_count % 10000 == 0:
            print('{} articles processed'.format(article_count))
    print('total: {} articles'.format(article_count))
finish_time = time.time()
print('Elapsed time: {}'.format(timedelta(seconds=finish_time-start_time)))

In [ ]:
#pembuatan model word2vec
start_time = time.time()
print('Training Word2Vec Model...')
sentences = word2vec.LineSentence('idwiki.txt')
id_w2v = word2vec.Word2Vec(sentences, size=200, workers=multiprocessing.cpu_count()-1)
id_w2v.save('idwiki_word2vec.model')
finish_time = time.time()
print('Finished. Elapsed time: {}'.format(timedelta(seconds=finish_time-start_time)))

In [ ]:
#Cheking the model
id_w2v.wv.most_similar("tdk")

In [ ]:
#Memproses data slang/alay
import pandas as pd
slang_corpus = pd.read_csv("colloquial-indonesian-lexicon.csv")
slang_corpus = slang_corpus[slang_corpus.columns[0:2]]
slang_corpus

In [ ]:
#Menggunakan perbandingan dengan word2vec yang ada dan edit distance untuk membuat kata menjadi kata formal
print("Processing data...")
start_time = time.time()
import nltk
corr_comments = []
slang_words = []
reco_words = []
corr_word = 0
tot_word = 0
for comment in tok_comments:
    corr_comment = []
    slang_word = []
    for sub_comment in comment:
        if sub_comment in id_w2v.wv.vocab:
            sim_slang = slang_corpus[slang_corpus['slang'] == sub_comment]
            if len(sim_slang) != 0:
                mode_word = sim_slang.formal.mode()[0]
                corr_word = corr_word + 1
                corr_comment.append(mode_word)
            else:
                corr_word = corr_word + 1
                corr_comment.append(sub_comment)
        else:
            comment_found = False
            sim_slang = slang_corpus[slang_corpus['slang'] == sub_comment]
            if len(sim_slang) != 0:
                mode_word = sim_slang.formal.mode()[0]
                corr_word = corr_word + 1
                corr_comment.append(mode_word)
                comment_found = True
            else:
                rat_words = []
                for slang in slang_corpus['slang']:
                    rat_word = nltk.edit_distance(sub_comment, slang)
                    rat_words.append(rat_word)
                min_rat = rat_words.index(min(rat_words))
                value = str(slang_corpus.formal.loc[min_rat])
                corr_words = corr_word + 1
                corr_comment.append(value)
                slang_words.append(sub_comment + " = " + value)
                comment_found = True
            if comment_found ==  False:
                corr_comment.append(sub_comment)
                slang_words.append(sub_comment + " = " + "Not Found")
        tot_word = tot_word + 1
    corr_comments.append(corr_comment)
    print("Total kata yang sudah diproses : " + str(tot_word))
print("Total kata yang diperbaiki : " + str(corr_word) + " dari total semua kata : " + str(tot_word))
finish_time = time.time()
print('Elapsed time: {}'.format(timedelta(seconds=finish_time-start_time)))

In [ ]:
#komentar yang sudah diperbaiki
corr_comments

In [ ]:
#Kata singkatan yang tidak ada pada slang corpus dan diolah dengan edit distance
slang_words